# Maximização de rendimento agrícola por algoritmos genéticos

## Rendimento previsto por floresta aleatória

Grupo: Emelyn Alves, João O. de A. Nascimento, Kayllany L. da S. Oliveira
<br>
Instituição: Ilum - Escola de Ciência
<br>
Professor: Daniel R. Cassar
<br>
Disciplina: Redes Neurais e Algoritmos Genéticos

## Importando bibliotecas

Inicialmente, é necessária a importação de bibliotecas e/ou módulos. Isso garante que todas as dependências e ferramentas essenciais estejam disponíveis para execução correta e eficiente do código. 

In [4]:
import pandas as pd
import random

from scipy.spatial.distance import cdist

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

import numpy as np
from deap import base
from deap import tools
from deap import creator
from deap.algorithms import eaSimple

## Tratando conjunto de dados

Agora, carregamos nosso conjunto de dados por meio da biblioteca `pandas`. A partir dessa tabela será possível criar um modelo capaz de prever as melhores condição para ter um bom rendimento (saiba mais sobre o dataset no README).

Importante ressaltar: os dados tabulados foram obtidos já separados em conjunto de treino (agricultural_yield_teste.csv) e teste (agricultural_yield_train.csv).

In [5]:
features = ['Soil_Quality', 'Seed_Variety', 'Fertilizer_Amount_kg_per_hectare', 'Sunny_Days', 'Rainfall_mm', 'Irrigation_Schedule']
target = ['Yield_kg_per_hectare']

df_teste = pd.read_csv("agricultural_yield_test.csv")
df_treino = pd.read_csv("agricultural_yield_train.csv")

df_teste = df_teste.reindex(features + target, axis=1) 
df_teste = df_teste.dropna()

df_treino = df_treino.reindex(features + target, axis=1)
df_treino = df_treino.dropna()

Após importar dados e remover valores ausentes, podemos formar conjuntos de dados de treino e teste.

## Conjuntos de dados de treino e teste da floresta aleatória 

Guardando os dados sem modificação para estudo da floresta aleatória: 

In [6]:
X_treino_floresta = df_treino.reindex(features, axis=1).values
y_treino_floresta = df_treino.reindex(target, axis=1).values.ravel()

X_teste_floresta = df_teste.reindex(features, axis=1).values
y_teste_floresta = df_teste.reindex(target, axis=1).values.ravel()

### Dados de treino

In [7]:
X_treino_floresta

array([[ 96.4156574 ,   1.        , 147.85304019,  94.59392594,
        444.26756885,   3.        ],
       [ 92.35262586,   0.        , 281.56539608,  90.5046436 ,
        517.58549117,   7.        ],
       [ 63.71478519,   1.        , 137.86493988,  97.32934017,
        420.3109448 ,   8.        ],
       ...,
       [ 67.47848719,   1.        , 120.01712248, 102.30862733,
        514.3727475 ,   8.        ],
       [ 85.17689046,   1.        , 247.84738886, 114.95663392,
        695.03580096,   6.        ],
       [ 87.8382803 ,   0.        , 244.21653603,  98.31373807,
        590.79491539,   6.        ]])

In [8]:
y_treino_floresta

array([683.75911882, 678.71486056, 934.69197491, ..., 932.82956077,
       889.7384376 , 554.69838897])

### Dados de teste 

In [9]:
X_teste_floresta

array([[ 93.30472076,   0.        , 132.52221761,  96.67092181,
        602.38623718,   3.        ],
       [ 83.67465324,   1.        ,  57.28399706,  99.00755589,
        466.51825061,   8.        ],
       [ 65.96303292,   1.        , 227.89547903, 104.8442723 ,
        510.32049549,   4.        ],
       ...,
       [ 94.14552167,   1.        , 196.47370358, 110.00347356,
        458.24140783,   3.        ],
       [ 61.45212028,   1.        ,  80.61835714,  97.81551671,
        778.6525395 ,   4.        ],
       [ 55.19590628,   0.        , 122.47323884,  90.31172043,
        582.99900253,   4.        ]])

In [10]:
y_teste_floresta

array([278.98656345, 836.43484043, 785.88178724, ..., 776.01363423,
       502.31948382, 257.03254443])

## Conversão de atributos em genes

Célula abaixo transforma cada atributo do dataset em listas separadas para serem usados como genes: 

In [11]:
qualidade_solo = list(df_teste['Soil_Quality'])
variedade_semente = list(df_teste['Seed_Variety'])
fertilizante = list(df_teste['Fertilizer_Amount_kg_per_hectare'])
dias_de_sol = list(df_teste['Sunny_Days'])
chuva = list(df_teste['Rainfall_mm'])
irrigação = list(df_teste['Irrigation_Schedule'])

rendimento = list(df_teste['Yield_kg_per_hectare'])

Abaixo, é criada uma lista que contém as listas geradas na célula anterior e obtendo os valores mínimos e máximos delas para criar uma nova lista com esses extremos. 

In [12]:
lista = [qualidade_solo, variedade_semente, fertilizante, dias_de_sol, chuva, irrigação]

min_max = []

for i in lista:
    min_max.append(min(i))
    min_max.append(max(i))

print("Tamanho lista:", len(min_max)) # Deve ser 6 x 2
print(min_max)

Tamanho lista: 12
[50.00362248192461, 99.99889758903616, 0, 1, 50.06265491148749, 299.9920540908917, 66.72446890908668, 138.52020171941672, 102.0075178449252, 876.694216751489, 0, 15]


## Floresta Aleatória

As células abaixo contêm a árvore aleatória que foi criada para testar o algoritmo.

In [13]:
SEMENTE_ALEATORIA = 61455

modelo_fa = RandomForestRegressor(random_state=SEMENTE_ALEATORIA)

In [14]:
modelo_fa.fit(X_treino_floresta, y_treino_floresta)

y_verdadeiro = y_teste_floresta
y_previsao = modelo_fa.predict(X_teste_floresta)
    
RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo árvore de decisão foi de {RMSE} unidades de y.")

O RMSE do modelo árvore de decisão foi de 54.89847461072623 unidades de y.


Testando o modelo para o maior rendimento do dataframe de teste: 

In [15]:
num = max(rendimento)

indice = rendimento.index(num)

linha = indice
individuo = df_treino.iloc[linha,:-1].values

print(individuo)
print(rendimento[indice])

[ 51.99126884   0.          66.6478779  100.72440418 454.76365482
   4.        ]
1406.1107054578508


In [16]:
print(individuo)
y_previsto = modelo_fa.predict([individuo])

print(y_previsto)

[ 51.99126884   0.          66.6478779  100.72440418 454.76365482
   4.        ]
[384.92875313]


### Usando módulo DEAP para algoritmo genético

Definindo hiperparâmetros: 

In [17]:
TAMANHO_POPULACAO = 100
NUM_GERACOES = 150
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.5
TAMANHO_TORNEIO = 5
TAMANHO_HALL_DA_FAMA = 1

Construindo um conjunto de ferramentas:

In [18]:
toolbox = base.Toolbox()

Observando mínimos e máximos de cada atributo do conjunto de dados:

In [19]:
i = 0
j = 0

for _ in range(6):
    print(features[i], '-> Min =', min_max[j], '/ Max =', min_max[j+1])
    print()
    i += 1
    j += 2

Soil_Quality -> Min = 50.00362248192461 / Max = 99.99889758903616

Seed_Variety -> Min = 0 / Max = 1

Fertilizer_Amount_kg_per_hectare -> Min = 50.06265491148749 / Max = 299.9920540908917

Sunny_Days -> Min = 66.72446890908668 / Max = 138.52020171941672

Rainfall_mm -> Min = 102.0075178449252 / Max = 876.694216751489

Irrigation_Schedule -> Min = 0 / Max = 15



Imprimindo a lista onde todos eles estão armazenados:

In [20]:
print(min_max)

[50.00362248192461, 99.99889758903616, 0, 1, 50.06265491148749, 299.9920540908917, 66.72446890908668, 138.52020171941672, 102.0075178449252, 876.694216751489, 0, 15]


Definição de uma função para cada gene:

* Esse passo foi realizado com o objetivo de restringir as possíveis escolhas que poderiam ser realizadas pelo módulo random. Assim, os valores obtidos estarão entre os valores mínimos e mácimos do conjunto de dados sintéticos original. 

In [21]:
def gene_soil_quality():
    return random.uniform(min_max[0], min_max[1])

def gene_seed_variety():
    return random.choice([min_max[2], min_max[3]])

def gene_fertilizer_amount():
    return random.uniform(min_max[4], min_max[5])

def gene_sunny_days():
    return random.uniform(min_max[6], min_max[7])

def gene_rainfall():
    return random.uniform(min_max[8], min_max[9])

def gene_irrigation_schedule():
    return random.randint(int(min_max[10]), int(min_max[11]))

Registrando os genes no toolbox

In [22]:
toolbox.register("gene_soil_quality", gene_soil_quality)
toolbox.register("gene_seed_variety", gene_seed_variety)
toolbox.register("gene_fertilizer_amount", gene_fertilizer_amount)
toolbox.register("gene_sunny_days", gene_sunny_days)
toolbox.register("gene_rainfall", gene_rainfall)
toolbox.register("gene_irrigation_schedule", gene_irrigation_schedule)

Criação de duas classes, uma para maximização e outra para a geração dos indivíduos.

In [23]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individuo", list, fitness=creator.FitnessMax)

Estabelecendo a função para a criação dos nossos indivíduos.

In [24]:
def cria_individuo():
    return creator.Individuo([
        toolbox.gene_soil_quality(),
        toolbox.gene_seed_variety(),
        toolbox.gene_fertilizer_amount(),
        toolbox.gene_sunny_days(),
        toolbox.gene_rainfall(),
        toolbox.gene_irrigation_schedule()
    ])

Testando a função de criação de indivíduos:

In [25]:
individuo_teste = cria_individuo()

print(individuo_teste)

[55.52971135391702, 0, 65.25876191367958, 98.11646742245712, 638.8355518612814, 13]


Definindo a função que especifica o tipo de mutação de cada gene: 

In [26]:
def mutacao_personalizada(individuo, chance_de_mutacao, min_max_list):
    
    min_max = min_max_list
        
    for gene in range(len(individuo)):
        if random.random() < chance_de_mutacao:
            if gene == 0:
                individuo[gene] = random.uniform(min_max[0], min_max[1])
            elif gene == 1:
                individuo[gene] = 0 if individuo[gene] == 1 else 1
            elif gene == 2:
                individuo[gene] = random.uniform(min_max[4], min_max[5])
            elif gene == 3:
                individuo[gene] = random.uniform(min_max[6], min_max[7])
            elif gene == 4:
                individuo[gene] = random.uniform(min_max[8], min_max[9])
            elif gene == 5:
                individuo[gene] = random.randint(int(min_max[10]), int(min_max[11]))
        
    return (individuo, )

Criação e teste da função objetivo usada em nosso algoritmo:

In [27]:
def funcao_objetivo_floresta(individuo):
    
    # Realizar a predição
    individuo = np.array(individuo).reshape(1, -1)  # Adicionar dimensão de batch
    y_previsao = modelo_fa.predict(individuo)
    
    # Calcular a distância de Manhattan entre y_pred e todos os valores de y_true 
    y_prev_val = y_previsao[0]
    y_true_vals = y_teste_floresta.reshape(-1, 1)
    distances = cdist([[y_prev_val]], y_true_vals, metric='cityblock')
    
    # Verificar a menor distância
    min_distance = distances.min()
    if min_distance > 500:
        y_prev_val = 0
    
    return (y_prev_val, )

Nesta célula, a função objetivo é adicionada como uma ferramenta de evolução na caixa de ferramentas.

In [28]:
toolbox.register("evaluate", funcao_objetivo_floresta)

Nesta célula, algumas funções e parâmetros são armazenados na caixa de ferramentas. Além disso, ela cria o "hall da fama", as estatísticas e a população inicial do algoritmo.

In [29]:
toolbox.register("cria_individuo", cria_individuo)
toolbox.register("populacao", tools.initRepeat, list, toolbox.cria_individuo, TAMANHO_POPULACAO)
toolbox.register("select", tools.selTournament, tournsize=TAMANHO_TORNEIO)
toolbox.register("mate", tools.cxUniform, indpb=0.5)

toolbox.register(
    "mutate",
    mutacao_personalizada,
    chance_de_mutacao = CHANCE_DE_MUTACAO,
    min_max_list = min_max
)

hall_da_fama_FA = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas_FA = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas_FA.register("média", np.mean)
estatisticas_FA.register("desv. padrão", np.std)
estatisticas_FA.register("min", np.min)
estatisticas_FA.register("max", np.max)

populacao_inicial_FA = toolbox.populacao()

Esta célula executa o algoritmo várias vezes e cria uma população final com os indivíduos do "hall da fama".

In [30]:
populacao_final_FA, log_FA = eaSimple(
    populacao_inicial_FA,
    toolbox,
    cxpb=CHANCE_DE_CRUZAMENTO,
    mutpb=CHANCE_DE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas_FA,
    halloffame=hall_da_fama_FA,
    verbose=True,
)

gen	nevals	média  	desv. padrão	min    	max   
0  	100   	808.313	267.632     	227.156	1325.6
1  	74    	1009.87	246.008     	243.292	1325.74
2  	77    	1059.49	270.092     	223.321	1329.87
3  	72    	1060.62	268.17      	424.178	1329.87
4  	72    	1146.53	246.233     	320.325	1334.03
5  	81    	1133.17	249.777     	352.368	1333.08
6  	76    	1078.74	310.56      	269.61 	1337.3 
7  	77    	1164.19	245.255     	427.944	1337.3 
8  	76    	1129.75	271.391     	269.839	1342.09
9  	72    	1184.39	219.8       	531.812	1345.01
10 	74    	1181.06	230.656     	312.563	1345.09
11 	78    	1144.39	272.575     	137.32 	1345.09
12 	78    	1168.42	245.162     	426.455	1345.09
13 	68    	1166.39	250.173     	280.63 	1345.09
14 	85    	1168.07	241.325     	458.785	1345.09
15 	76    	1170.75	245.881     	476.425	1345.09
16 	67    	1183.95	241.77      	406.33 	1345.09
17 	73    	1198.99	243.613     	185.375	1345.09
18 	73    	1163.36	258.958     	337.289	1345.09
19 	86    	1107.25	297.134     	118.465	13

Imprime os parâmetros do vencedor: 

In [31]:
print(hall_da_fama_FA.items)

[[70.16680388561842, 1, 292.7155314314829, 115.84271791206353, 165.8286567904463, 14]]


Comparação entre indivíduo vencedor com o valor previsto de y:

In [ ]:
hall_da_fama_FA = np.array(hall_da_fama_FA).reshape(1, -1)
print(hall_da_fama_FA)
y_previsao = modelo_fa.predict(hall_da_fama_FA)
    
print(f"A previsão máxima de rendimento encontrada foi: {y_previsao[0]}")

[[ 70.16680389   1.         292.71553143 115.84271791 165.82865679
   14.        ]]
A previsão máxima de rendimento encontrada foi: 1351.8934062229985
